In [3]:
import sys
sys.path.insert(0, '..')

In [4]:
import torch


print(torch.__version__)
print("GPU Available:", torch.cuda.is_available())

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = "cpu"

2.0.1
GPU Available: False


In [5]:
import torch.nn as nn
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
batch_size = 64

torch.manual_seed(1)
np.random.seed(1)

In [7]:
import pickle

# To load the lists from the file:
with open(r'C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\EEG-To-Text-GAN\Data\EEG_Text_Pairs.pkl', 'rb') as file:
    EEG_word_level_embeddings = pickle.load(file)
    EEG_word_level_labels = pickle.load(file)

In [8]:
float_tensor = torch.tensor(EEG_word_level_embeddings, dtype=torch.float)
float_tensor = float_tensor.unsqueeze(1)

C:\Users\gxb18167\AppData\Local\Temp\ipykernel_21112\2008157655.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\b\abs_abjetg6_iu\croot\pytorch_1686932924616\work\torch\csrc\utils\tensor_new.cpp:248.)
  float_tensor = torch.tensor(EEG_word_level_embeddings, dtype=torch.float)


In [9]:
float_tensor.shape


torch.Size([63015, 1, 105, 8])

In [10]:
import torch
train_data = []
for i in range(len(float_tensor)):
   train_data.append([float_tensor[i], EEG_word_level_labels[i]])



trainloader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=64)

In [11]:
#sanity check:
i1, l1 = next(iter(trainloader))
print(i1.shape, l1)

torch.Size([64, 1, 105, 8]) ('campaign', 'the', '16', 'returned', 'politics', 'left', '1591', 'singers.', 'cancer', 'backing', 'continued', 'Springwells', 'accession', 'not,', 'Pentagon', 'shirt', '(born', 'Lionel', 'for', 'stood', 'Whipple', 'Ford', 'church', 'Pennsylvania.', '1986', 'Finally,', 'C.H.', 'married', 'Friends.', 'publisher', 'a', 'President', 'de', 'Wagner,', 'free', 'and', 'interrupted', 'from', 'again', "Charles's", 'experiments', 'marriage', 'has', 'Republican', '14', 'Presidential', 'trial', 'his', 'whom', 'President', 'the', 'to', 'led', 'Her', 'trained', 'York', 'Captain', 'primary', 'pedophile,', 'his', 'degree', 'erroneously', 'cgs', 'an')


In [12]:
z_size = 100
image_size = (105, 8)
n_filters = 32

In [13]:
## Loss function and optimizers:
loss_fn = nn.BCELoss()

In [14]:
def create_noise(batch_size, z_size, mode_z):
    if mode_z == 'uniform':
        input_z = torch.rand(batch_size, z_size)*2 - 1
    elif mode_z == 'normal':
        input_z = torch.randn(batch_size, z_size)
    return input_z

mode_z = 'uniform'
fixed_z = create_noise(batch_size, z_size, mode_z).to(device)

def create_samples(g_model, input_z):
    g_output = g_model(input_z)
    images = torch.reshape(g_output, (batch_size, *image_size))
    return (images+1)/2.0

In [15]:
noise = create_noise(64, 100, "uniform")

In [16]:
noise.shape

torch.Size([64, 100])

In [17]:
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, latent_dim=100):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim

        # Define your layers here
        self.fc = nn.Linear(latent_dim, 1 * 105 * 8)
        self.conv_transpose = nn.ConvTranspose2d(1, 1, kernel_size=3, stride=1, padding=1)

    def forward(self, z):
        # Reshape the input
        x = self.fc(z)
        x = x.view(-1, 1, 105, 8)

        # Apply convolutional transpose layer
        x = self.conv_transpose(x)

        return x



class DiscriminatorWGAN(nn.Module):
    def __init__(self, n_filters):
        super(DiscriminatorWGAN, self).__init__()
        self.network = nn.Sequential(
            nn.Conv2d(1, n_filters, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2),

            nn.Conv2d(n_filters, n_filters*2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.InstanceNorm2d(n_filters * 2),
            nn.LeakyReLU(0.2),

            nn.Conv2d(n_filters*2, n_filters*4, kernel_size=3, stride=2, padding=1, bias=False),
            nn.InstanceNorm2d(n_filters*4),
            nn.LeakyReLU(0.2),

            nn.Conv2d(n_filters*4, 1, kernel_size=3, stride=1, padding=1, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        output = self.network(input)
        return output.view(-1, 1).squeeze(0)



In [18]:
gen_model = Generator().to(device)
disc_model = DiscriminatorWGAN(n_filters).to(device)

g_optimizer = torch.optim.Adam(gen_model.parameters(), 0.0002)
d_optimizer = torch.optim.Adam(disc_model.parameters(), 0.0002)

In [19]:
from torch.autograd import grad as torch_grad


def gradient_penalty(real_data, generated_data):
    batch_size = real_data.size(0)

    # Calculate interpolation
    alpha = torch.rand(real_data.shape[0], 1, 1, 1, requires_grad=True, device=device)
    print("Gen:", generated_data.shape)
    interpolated = alpha * real_data + (1 - alpha) * generated_data

    # Calculate probability of interpolated examples
    proba_interpolated = disc_model(interpolated)

    # Calculate gradients of probabilities with respect to examples
    gradients = torch_grad(outputs=proba_interpolated, inputs=interpolated,
                           grad_outputs=torch.ones(proba_interpolated.size(), device=device),
                           create_graph=True, retain_graph=True)[0]

    gradients = gradients.view(batch_size, -1)
    gradients_norm = gradients.norm(2, dim=1)
    return lambda_gp * ((gradients_norm - 1)**2).mean()


In [20]:
## Train the discriminator
def d_train_wgan(x):
    disc_model.zero_grad()

    batch_size = x.size(0)
    x = x.to(device)

    # Calculate probabilities on real and generated data
    d_real = disc_model(x)
    input_z = create_noise(batch_size, z_size, mode_z).to(device)
    print("Input Z:", input_z.shape)
    g_output = gen_model(input_z)
    print("G:", g_output.shape)

    d_generated = disc_model(g_output)
    d_loss = d_generated.mean() - d_real.mean() + gradient_penalty(x.data, g_output.data)

    d_loss.backward()
    d_optimizer.step()

    return d_loss.data.item()

In [21]:
## Train the generator
def g_train_wgan(x):
    gen_model.zero_grad()

    batch_size = x.size(0)
    input_z = create_noise(batch_size, z_size, mode_z).to(device)

    g_output = gen_model(input_z)

    d_generated = disc_model(g_output)
    g_loss = -d_generated.mean()

    # gradient backprop & optimize ONLY G's parameters
    g_loss.backward()
    g_optimizer.step()

    return g_loss.data.item()

In [22]:
epoch_samples_wgan = []
lambda_gp = 10.0
num_epochs = 100
torch.manual_seed(1)
critic_iterations = 5

for epoch in range(1, num_epochs+1):
    gen_model.train()
    d_losses, g_losses = [], []
    for i, (x, _) in enumerate(trainloader):
        print("Batch:", i)
        print("X:", x.shape)
        for _ in range(critic_iterations):
            d_loss = d_train_wgan(x)
        d_losses.append(d_loss)
        g_losses.append(g_train_wgan(x))

    print(f'Epoch {epoch:03d} | D Loss >>'
          f' {torch.FloatTensor(d_losses).mean():.4f}')
    gen_model.eval()
    epoch_samples_wgan.append(
        create_samples(gen_model, fixed_z).detach().cpu().numpy())

Batch: 0
X: torch.Size([64, 1, 105, 8])
Input Z: torch.Size([64, 100])
G: torch.Size([64, 1, 105, 8])
Gen: torch.Size([64, 1, 105, 8])
Input Z: torch.Size([64, 100])
G: torch.Size([64, 1, 105, 8])
Gen: torch.Size([64, 1, 105, 8])
Input Z: torch.Size([64, 100])
G: torch.Size([64, 1, 105, 8])
Gen: torch.Size([64, 1, 105, 8])
Input Z: torch.Size([64, 100])
G: torch.Size([64, 1, 105, 8])
Gen: torch.Size([64, 1, 105, 8])
Input Z: torch.Size([64, 100])
G: torch.Size([64, 1, 105, 8])
Gen: torch.Size([64, 1, 105, 8])
Batch: 1
X: torch.Size([64, 1, 105, 8])
Input Z: torch.Size([64, 100])
G: torch.Size([64, 1, 105, 8])
Gen: torch.Size([64, 1, 105, 8])
Input Z: torch.Size([64, 100])
G: torch.Size([64, 1, 105, 8])
Gen: torch.Size([64, 1, 105, 8])
Input Z: torch.Size([64, 100])
G: torch.Size([64, 1, 105, 8])
Gen: torch.Size([64, 1, 105, 8])
Input Z: torch.Size([64, 100])
G: torch.Size([64, 1, 105, 8])
Gen: torch.Size([64, 1, 105, 8])
Input Z: torch.Size([64, 100])
G: torch.Size([64, 1, 105, 8])
Gen

KeyboardInterrupt: 